<a href="https://colab.research.google.com/github/ced-sys/SubTerra/blob/main/Negative_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import geopandas as gpd
from shapely.geometry import Point
import random

gpd.options.io_engine = "pyogrio"


# Paths
kenya_path = '/content/drive/MyDrive/SubTerra/kenya_boundaries.shp'
buffer_path = '/content/drive/MyDrive/SubTerra/Data For ML/positive_buffer.geojson'
output_path = '/content/drive/MyDrive/SubTerra/Data For ML/negative_samples.geojson'

# Load Kenya boundary shapefile
kenya = gpd.read_file(kenya_path)
buffer_zones = gpd.read_file(buffer_path)

# Assign initial CRS (assuming WGS84)
kenya = kenya.set_crs(epsg=4326)
buffer_zones = buffer_zones.set_crs(epsg=4326)

# Reproject to UTM (meters)
kenya = kenya.to_crs(epsg=32636)
buffer_zones = buffer_zones.to_crs(epsg=32636)

# Kenya bounds for random sampling
minx, miny, maxx, maxy = kenya.total_bounds

# Generate random negative points
points = []
labels = []
target = 100  # Number of negative points
attempts = 0
max_attempts = 10000

while len(points) < target and attempts < max_attempts:
    x = random.uniform(minx, maxx)
    y = random.uniform(miny, maxy)
    pnt = Point(x, y)
    if kenya.geometry.unary_union.contains(pnt) and not buffer_zones.geometry.unary_union.intersects(pnt):
        points.append(pnt)
        labels.append(0)
    attempts += 1

# Create GeoDataFrame for negative samples
neg_gdf = gpd.GeoDataFrame({'label': labels}, geometry=points, crs='EPSG:32636')
neg_gdf = neg_gdf.to_crs(epsg=4326)

# Save as GeoJSON
neg_gdf.to_file(output_path, driver='GeoJSON')

print(f"✅ Generated {len(neg_gdf)} negative points outside geothermal buffers.")

<ipython-input-11-caefab8aa8b5>:39: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if kenya.geometry.unary_union.contains(pnt) and not buffer_zones.geometry.unary_union.intersects(pnt):
<ipython-input-11-caefab8aa8b5>:39: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if kenya.geometry.unary_union.contains(pnt) and not buffer_zones.geometry.unary_union.intersects(pnt):
<ipython-input-11-caefab8aa8b5>:39: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if kenya.geometry.unary_union.contains(pnt) and not buffer_zones.geometry.unary_union.intersects(pnt):
<ipython-input-11-caefab8aa8b5>:39: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if kenya.geometry.unary_union.contains(pnt) and not buffer_zones.geometry.unary_union.intersects(pnt):
<ipython-input-11-caefab8aa8b5>:

✅ Generated 100 negative points outside geothermal buffers.
